Import Library

In [3]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), "SwinIR")) # setting directory to ensure it recognizes the SwinIR github folder

In [4]:
import torch
import random
import shutil
from PIL import Image

from basicsr.archs.swinir_arch import SwinIR # if this throws an error, try re running block 1 again
import torch.nn.functional as F

from torchvision import transforms, datasets

Transform

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), # resize image to 224 x 224
    transforms.ToTensor(), # convert image to tensor [0, 1]
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3) # normalize to [-1, 1] for CNN
])

Load Full Dataset

In [6]:
full_dataset = datasets.ImageFolder(
    root="data",
    transform=transform
)
print("Class to index mapping:", full_dataset.class_to_idx)
len(full_dataset)

Class to index mapping: {'COVID': 0, 'Normal': 1, 'Viral Pneumonia': 2}


15153

Sample the images

In [7]:
input_directory = "data"
output_directory = "data_subset"
subset_size = 500

if os.path.exists(output_directory): # if the output directory exist, override it
    shutil.rmtree(output_directory)
os.makedirs(output_directory)

all_images = []

for i in os.listdir(input_directory): # for every classes folder (COVID, Normal, Viral Pneumonia)
    class_path = os.path.join(input_directory, i) # combine into one directory: the root directory with the class directory
    if not os.path.isdir(class_path): # if it's not a directory, pass
        continue

    for j in os.listdir(class_path): # for every file in the classes folder
        if j.lower().endswith(('.jpg', '.jpeg', '.png')): # if the file is an image
            full_path = os.path.join(class_path, j) # combine into one directory: the full class directory and the image directory
            all_images.append((full_path, i)) # append to all_images list

sampled_images = random.sample(all_images, subset_size) # random sample x images from the list

for path, class_name in sampled_images:
    destination_folder = os.path.join(output_directory, class_name) # new directory path
    os.makedirs(destination_folder, exist_ok=True) # make the directory
    shutil.copy(path, os.path.join(destination_folder, os.path.basename(path))) # copy the file from the original input_directory into the output_directory

print(f"Subset of {subset_size} images completed. Saved to {output_directory}")

Subset of 500 images completed. Saved to data_subset


Downsampling (reduce resolution) Images

In [8]:
input_directory = "data_subset"
output_directory = "data_subset_lowres"
target_size = (56, 56)

for i in os.listdir(input_directory): # for every classes folder (COVID, Normal, Viral Pneumonia)
    class_path = os.path.join(input_directory, i) # identify the current path
    destination_path = os.path.join(output_directory, i) # identify the destination path
    
    if not os.path.isdir(class_path): # if it is not a path, pass
        continue
    
    if os.path.exists(destination_path): # if the destination path already existed, override its content
        shutil.rmtree(destination_path) 
    os.makedirs(destination_path) # make the directory

    for j in os.listdir(class_path): # for every file in the classes folder
        if not j.endswith('.jpg') and not j.endswith('.png'): # if the file is not an image, pass
            continue

        image_path = os.path.join(class_path, j) # create a new path
        image = Image.open(image_path).convert("RGB") # open the image file and store it as image variable
        image = image.resize(target_size, resample=Image.BICUBIC) # resize
        image.save(os.path.join(destination_path, j)) # save the image

print("Downsampling complete. Saved to 'data_subset_lowres'")

Downsampling complete. Saved to 'data_subset_lowres'


SwinIR Image Super Resolution

In [9]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") # trying mps since I'm using a mac

swinir_model = SwinIR(
    upscale=4, # we want to upscale 4 times (from downscaled 56x56 to the original 224x224)
    in_chans=3, # default 3 channels for RGB
    img_size=48, # based on paper
    window_size=8, # based on paper
    img_range=1.0, # pixel range of the images [0, 1]
    depths=[6, 6, 6, 6, 6, 6], # according to the pretrained model github
    embed_dim=180, # according to the pretrained model github
    num_heads=[6, 6, 6, 6, 6, 6], # according to the pretrained model github
    mlp_ratio=2, # default
    upsampler='pixelshuffle', # default
    resi_connection='1conv' # default
)

weights_path = "SwinIR/model_zoo/001_classicalSR_DIV2K_s48w8_SwinIR-M_x4.pth" # pretrain model weights
checkpoint = torch.load(weights_path)
swinir_model.load_state_dict(checkpoint['params'], strict=True)
swinir_model.eval().to(device)

to_tensor = transforms.ToTensor() # PIL to Tensor
to_pil = transforms.ToPILImage() # Tensor to PIL

input_directory = "data_subset_lowres"
output_directory = "data_subset_swinir"
window_size = 8 # required by SwinIR to be divisible by 8

for i in os.listdir(input_directory): # for every classes folder (COVID, Normal, Viral Pneumonia)
    class_path = os.path.join(input_directory, i) # identify the path
    destination_path = os.path.join(output_directory, i) # identify the destination path

    if not os.path.isdir(class_path): # if it's not a directory, pass
        continue
    
    if os.path.exists(destination_path): # if the destination path already existed, override its content 
        shutil.rmtree(destination_path) 
    os.makedirs(destination_path) # make the directory

    for j in os.listdir(class_path): # for every file in the classes folder
        if not j.lower().endswith(('.jpg', '.png', '.jpeg')): # if it's not an image, pass
            continue

        image = Image.open(os.path.join(class_path, j)).convert("RGB") # store the current image into a variable
        image_tensor = to_tensor(image).unsqueeze(0).to(device)

        # padding
        _, _, h, w = image_tensor.size() 
        pad_h = (window_size - h % window_size) % window_size 
        pad_w = (window_size - w % window_size) % window_size
        image_tensor = F.pad(image_tensor, (0, pad_w, 0, pad_h), mode='reflect')

        with torch.no_grad():
            sr_tensor = swinir_model(image_tensor).clamp(0, 1) # running swinir

        swinir_image = to_pil(sr_tensor.squeeze(0).cpu())
        swinir_image.save(os.path.join(destination_path, j))

print("SwinIR complete. Saved to 'data_subset_swinir'") # should take around 2.5 - 3.5 minutes.

/opt/anaconda3/lib/python3.12/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_e2rjusv8ek/croot/pytorch-select_1717607459930/work/aten/src/ATen/native/TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SwinIR complete. Saved to 'data_subset_swinir'
